<a href="https://colab.research.google.com/github/JeongheeKim219/data_a_s_practice/blob/titanic/titanic_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Titanic 생존률 예측
---
*   출처: https://www.kaggle.com/competitions/titanic/code

1.   데이터 피처
  - PassengerId : 탑승객 Id
  - survived: 생존여부
  - pclass: 티켓클래스 (1st-upper, 2nd-middle, 3rd-lower)
  - Name: 이름
  - sex : 성별
  - Age : 나이 (소수점으로 표시될 수도 있음. 1보다 작을 경우 혹은 추측일 경우 xx.5 형태)
  - sibsp: 배에 탑승한 형제나 배우자의 수
  - parch: 부모나 자녀의 수 (일부 유모와 함께한 아이의 경우 parch 값은 0)
  - ticket: 티켓 넘버
  - fare: 탑승요금
  - cabin: 객실번호
  - embarked: 승선항, C = Cherbourg, Q = Queenstown, S = Southampton


2. 타겟값(예측해야하는 값)
  - PassengerId : 생존한 승객




## 1. 필수 라이브러리 로드

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 2. 데이터로드

In [ ]:
train = pd.read_csv("kaggle/input/titanic/train.csv")
test = pd.read_csv("kaggle/input/titanic/test.csv")

print(train.shape,test.shape)

(891, 12) (418, 11)


### 2-2. 데이터 개괄

*   전체 데이터 수: 891

*   전체 컬럼: 'PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'

*   결측치
  - Age(177/891, 19.8%)
  - Cabin(687/891, 77.1%)   
  - Embarked(2/891, 0.002%)

* 데이터 타입
  - 문자형 컬럼: 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'
  - 숫자형 컬럼: 'PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'



In [ ]:
# 전체 컬럼 정보
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
# 컬럼 데이터 타입
total_columns = train.columns.tolist()
object_columns = train.select_dtypes(include=["object"]).columns.tolist()
numeric_columns = [x for x in total_columns if x not in object_columns]

In [ ]:
print(f'전체 컬럼 : {[x for x in total_columns]}')
print(f'문자형 컬럼 : {object_columns}')
print(f'숫자형 컬럼 : {numeric_columns}')

전체 컬럼 : ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
문자형 컬럼 : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
숫자형 컬럼 : ['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']


In [ ]:
# 데이터 미리보기
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# 숫자형 데이터의 기초 통계 정보
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
# 문자열 데이터의 기초 통계 정보
train.describe(include = 'object')

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",male,347082,B96 B98,S
freq,1,577,7,4,644


In [ ]:
# 유독 unique 값이 많은 Ticket 컬럼을 살펴본다.
train['Ticket'].unique()

NameError: name 'train' is not defined